# Basic Walkthrough

In [1]:
cd ../../

In [2]:
import pandas as pd
import lux

We first load in the [Cars dataset](http://lib.stat.cmu.edu/datasets/) with 392 different cars from 1970-1982, which contains information about its Horsepower, MilesPerGal, Acceleration, etc.

In [3]:
df = pd.read_csv("lux/data/car.csv")
df["Year"] = pd.to_datetime(df["Year"], format='%Y') # change pandas dtype for the column "Year" to datetype

We print out the dataframe, we see the default Pandas display and we can toggle to a set of recommendations generated by Lux. 
Lux returns four sets of visualizations to show an overview of the dataset.


In [21]:
isinstance(vis.code,dict)

True

True

In [10]:
from lux.vis.Vis import Vis

In [13]:
vis = Vis(["Horsepower","Weight"],df)

In [25]:
assert isinstance(vis.min_max,dict)

In [28]:
df = pd.read_csv("lux/data/car.csv")
vis = Vis(["Horsepower","Weight"],df)
vis._repr_html_()
vis.code

LuxWidget(current_vis={'config': {'view': {'continuousWidth': 400, 'continuousHeight': 300}, 'axis': {'labelCo…

{'config': {'view': {'continuousWidth': 400, 'continuousHeight': 300},
  'axis': {'labelColor': '#505050',
   'labelFont': 'Helvetica Neue',
   'labelFontSize': 9,
   'labelFontWeight': 400,
   'titleFont': 'Helvetica Neue',
   'titleFontSize': 11,
   'titleFontWeight': 500},
  'legend': {'labelFont': 'Helvetica Neue',
   'labelFontSize': 9,
   'labelFontWeight': 400,
   'titleFont': 'Helvetica Neue',
   'titleFontSize': 10,
   'titleFontWeight': 500},
  'mark': {'tooltip': {'content': 'encoding'}},
  'title': {'font': 'Helvetica Neue', 'fontSize': 13, 'fontWeight': 500}},
 'data': {'name': 'data-d51965f82237e6af22c157040d8a1068'},
 'mark': 'circle',
 'encoding': {'x': {'type': 'quantitative',
   'field': 'Horsepower',
   'scale': {'domain': [46.0, 230.0]}},
  'y': {'type': 'quantitative',
   'field': 'Weight',
   'scale': {'domain': [1613.0, 5140.0]}}},
 'height': 150,
 'selection': {'selector015': {'type': 'interval',
   'bind': 'scales',
   'encodings': ['x', 'y']}},
 'width': 160,


In [29]:
vis.intent

['Horsepower', 'Weight']

Here we spot this scatterplot visualization `Acceleration` v.s. `Horsepower`.
Intuitively, we expect cars with higher horsepower to have higher acceleration, but we are actually seeing the opposite of that trend. 

Let's learn more about whether there are additional factors that is affecting this relationship.
Using the `intent` property, we indicate to Lux that we are interested in the attributes `Acceleration` and `Horsepower`.

In [5]:
df.intent = ["Acceleration","Horsepower"]
df

Button(description='Toggle Pandas/Lux', layout=Layout(top='5px', width='140px'), style=ButtonStyle())

Output()

On the left, we see that the Current Visualization corresponds to our specified intent.
On the right, we see different tabs of recommendations: 
- `Enhance` shows what happens when we add an additional variable to the current selection
- Then, we have the `Filter` tab, which adds an additional filter, while fixing the selected variables on the X Y axes
- Finally, we have `Generalize` which removes one of the attributes, to display the more general trend.


Button(description='Toggle Pandas/Lux', layout=Layout(top='5px', width='140px'), style=ButtonStyle())

Output()

In [10]:
vis

LuxWidget(current_vis={'config': {'view': {'continuousWidth': 400, 'continuousHeight': 300}, 'axis': {'labelCo…

<Vis  ([<Clause  channel: x,
         attribute: Acceleration,
         data_model: measure,
         data_type: quantitative,
         bin_size: 0 >, <Clause  channel: y,
         attribute: Horsepower,
         data_model: measure,
         data_type: quantitative,
         bin_size: 0 >, <Clause  channel: color,
         attribute: Origin,
         data_model: dimension,
         data_type: nominal,
         bin_size: 0 >]) mark: scatter, score: 0.3333333333333333 >

We can really quickly compare the relationship between `Horsepower` and `Acceleration` when an additional factor is in play. For example, we see here that `Displacement` and `Weight` is higher on the top left and lower towards the bottom right, whereas `MilesPerGal` has the opposite effect.

We see that there is a strong separation between cars that have 4 cylinders (Orange) and cars with 8 cylinders (green), so we are interested in learning more about the attribute `Cylinders`.

In [6]:
df.intent = ["Cylinders"]
df

Button(description='Toggle Pandas/Lux', layout=Layout(top='5px', width='140px'), style=ButtonStyle())

Output()

The Count distribution shows that there is not a lot of cars with 3 and 5 cylinders, so let's clean the data up to remove those.

In [7]:
df[df["Cylinders"]==3].to_pandas()

,Name,MilesPerGal,Cylinders,Displacement,Horsepower,Weight,Acceleration,Year,Origin
70,mazda rx2 coupe,19.0,3,70.0,97,2330,13.5,1972-01-01,Japan
110,maxda rx3,18.0,3,70.0,90,2124,13.5,1973-01-01,Japan
241,mazda rx-4,21.5,3,80.0,110,2720,13.5,1977-01-01,Japan
331,mazda rx-7 gs,23.7,3,70.0,100,2420,12.5,1980-01-01,Japan


In [8]:
df[df["Cylinders"]==5].to_pandas()


,Name,MilesPerGal,Cylinders,Displacement,Horsepower,Weight,Acceleration,Year,Origin
272,audi 5000,20.3,5,131.0,103,2830,15.9,1978-01-01,Europe
295,mercedes benz 300d,25.4,5,183.0,77,3530,20.1,1979-01-01,Europe
325,audi 5000s (diesel),36.4,5,121.0,67,2950,19.9,1980-01-01,Europe


Note how easy it is to clean up data and update recommendations in Lux, due to the tight integration with Pandas.

In [12]:
df = df[(df["Cylinders"]!=3) & (df["Cylinders"]!=5)]
df

Button(description='Toggle Pandas/Lux', layout=Layout(top='5px', width='140px'), style=ButtonStyle())

Output()

Let's say we find the time series showing the number of cars of different `Cylinder` over time to be very interesting. In particular, there seems to be a spike in production of 4 Cylinder cars in 1982. To dig into this more, we can export it by selecting the visualization and clicking on the export button.

In [14]:
vis = df.exported[0]
vis

LuxWidget(current_vis={'config': {'view': {'continuousWidth': 400, 'continuousHeight': 300}, 'axis': {'labelCo…

<Vis  ([<Clause  channel: x,
         attribute: Year,
         data_model: dimension,
         data_type: temporal,
         bin_size: 0 >, <Clause  channel: y,
         attribute: Record,
         aggregation: count,
         data_model: measure,
         data_type: quantitative,
         bin_size: 0 >]) mark: line, score: 0.15 >

We can then print out the visualization code in [Altair](https://altair-viz.github.io/) that generated this chart so that we can further tweak the chart as desired.

In [15]:
print(vis.to_Altair())

import altair as alt
import pandas._libs.tslibs.timestamps
from pandas._libs.tslibs.timestamps import Timestamp
visData = pd.DataFrame({'Year': {0: Timestamp('1970-01-01 00:00:00'), 1: Timestamp('1970-01-01 00:00:00'), 2: Timestamp('1970-01-01 00:00:00'), 3: Timestamp('1971-01-01 00:00:00'), 4: Timestamp('1971-01-01 00:00:00'), 5: Timestamp('1971-01-01 00:00:00'), 6: Timestamp('1972-01-01 00:00:00'), 7: Timestamp('1972-01-01 00:00:00'), 8: Timestamp('1972-01-01 00:00:00'), 9: Timestamp('1973-01-01 00:00:00'), 10: Timestamp('1973-01-01 00:00:00'), 11: Timestamp('1973-01-01 00:00:00'), 12: Timestamp('1974-01-01 00:00:00'), 13: Timestamp('1974-01-01 00:00:00'), 14: Timestamp('1974-01-01 00:00:00'), 15: Timestamp('1975-01-01 00:00:00'), 16: Timestamp('1975-01-01 00:00:00'), 17: Timestamp('1975-01-01 00:00:00'), 18: Timestamp('1976-01-01 00:00:00'), 19: Timestamp('1976-01-01 00:00:00'), 20: Timestamp('1976-01-01 00:00:00'), 21: Timestamp('1977-01-01 00:00:00'), 22: Timestamp('1977-01-01 00:

In [19]:
import altair as alt
import pandas._libs.tslibs.timestamps
from pandas._libs.tslibs.timestamps import Timestamp
visData = pd.DataFrame({'Year': {0: Timestamp('1970-01-01 00:00:00'), 1: Timestamp('1970-01-01 00:00:00'), 2: Timestamp('1970-01-01 00:00:00'), 3: Timestamp('1971-01-01 00:00:00'), 4: Timestamp('1971-01-01 00:00:00'), 5: Timestamp('1971-01-01 00:00:00'), 6: Timestamp('1972-01-01 00:00:00'), 7: Timestamp('1972-01-01 00:00:00'), 8: Timestamp('1972-01-01 00:00:00'), 9: Timestamp('1973-01-01 00:00:00'), 10: Timestamp('1973-01-01 00:00:00'), 11: Timestamp('1973-01-01 00:00:00'), 12: Timestamp('1974-01-01 00:00:00'), 13: Timestamp('1974-01-01 00:00:00'), 14: Timestamp('1974-01-01 00:00:00'), 15: Timestamp('1975-01-01 00:00:00'), 16: Timestamp('1975-01-01 00:00:00'), 17: Timestamp('1975-01-01 00:00:00'), 18: Timestamp('1976-01-01 00:00:00'), 19: Timestamp('1976-01-01 00:00:00'), 20: Timestamp('1976-01-01 00:00:00'), 21: Timestamp('1977-01-01 00:00:00'), 22: Timestamp('1977-01-01 00:00:00'), 23: Timestamp('1977-01-01 00:00:00'), 24: Timestamp('1978-01-01 00:00:00'), 25: Timestamp('1978-01-01 00:00:00'), 26: Timestamp('1978-01-01 00:00:00'), 27: Timestamp('1979-01-01 00:00:00'), 28: Timestamp('1979-01-01 00:00:00'), 29: Timestamp('1979-01-01 00:00:00'), 30: Timestamp('1980-01-01 00:00:00'), 31: Timestamp('1980-01-01 00:00:00'), 32: Timestamp('1980-01-01 00:00:00'), 33: Timestamp('1982-01-01 00:00:00'), 34: Timestamp('1982-01-01 00:00:00'), 35: Timestamp('1982-01-01 00:00:00')}, 'Cylinders': {0: 8, 1: 6, 2: 4, 3: 8, 4: 6, 5: 4, 6: 8, 7: 6, 8: 4, 9: 8, 10: 6, 11: 4, 12: 8, 13: 6, 14: 4, 15: 6, 16: 4, 17: 8, 18: 4, 19: 6, 20: 8, 21: 4, 22: 6, 23: 8, 24: 8, 25: 4, 26: 6, 27: 4, 28: 6, 29: 8, 30: 6, 31: 8, 32: 4, 33: 4, 34: 8, 35: 6}, 'Record': {0: 18.0, 1: 4.0, 2: 7.0, 3: 7.0, 4: 8.0, 5: 12.0, 6: 13.0, 7: 0.0, 8: 14.0, 9: 20.0, 10: 8.0, 11: 11.0, 12: 5.0, 13: 6.0, 14: 15.0, 15: 12.0, 16: 12.0, 17: 6.0, 18: 15.0, 19: 10.0, 20: 9.0, 21: 14.0, 22: 5.0, 23: 8.0, 24: 6.0, 25: 17.0, 26: 12.0, 27: 12.0, 28: 6.0, 29: 10.0, 30: 2.0, 31: 0.0, 32: 23.0, 33: 47.0, 34: 1.0, 35: 10.0}})

chart = alt.Chart(visData).mark_line().encode(
    y = alt.Y('Record', type= 'quantitative', title='Number of Records'),
    x = alt.X('Year', type = 'temporal'),
)
chart = chart.interactive() # Enable Zooming and Panning
chart = chart.encode(color=alt.Color('Cylinders',type='nominal'))

chart = chart.configure_title(fontWeight=500,fontSize=13,font='Helvetica Neue')
chart = chart.configure_axis(titleFontWeight=500,titleFontSize=11,titleFont='Helvetica Neue',
			labelFontWeight=400,labelFontSize=8,labelFont='Helvetica Neue',labelColor='#505050')
chart = chart.configure_legend(titleFontWeight=500,titleFontSize=10,titleFont='Helvetica Neue',
			labelFontWeight=400,labelFontSize=8,labelFont='Helvetica Neue')
chart = chart.properties(width=180,height=150)

chart

alt.Chart(...)

This is obviously a lot of code, let's look at how easy it is if we were to specify this visualization intent in Lux if we knew what we were looking for.

In [20]:
from lux.vis.Vis import Vis
Vis(["Year","Cylinders"],df)

LuxWidget(current_vis={'config': {'view': {'continuousWidth': 400, 'continuousHeight': 300}, 'axis': {'labelCo…

<Vis  (x: Year, y: COUNT(Record), color: Cylinders) mark: line, score: 0.0 >

# Creating Visualizations 

In Lux, user can specify particular visualizations that they want to specify and visualize their data on-demand.

In [21]:
from lux.vis.Vis import Vis

In [22]:
Vis(["Horsepower"],df)

LuxWidget(current_vis={'config': {'view': {'continuousWidth': 400, 'continuousHeight': 300}, 'axis': {'labelCo…

<Vis  (x: BIN(Horsepower), y: COUNT(Record)) mark: histogram, score: 0.0 >

In [23]:
Vis(["Origin","Horsepower"],df)

LuxWidget(current_vis={'config': {'view': {'continuousWidth': 400, 'continuousHeight': 300}, 'axis': {'labelCo…

<Vis  (x: MEAN(Horsepower), y: Origin) mark: bar, score: 0.0 >

In [24]:
Vis(["Origin",lux.Clause("Horsepower",aggregation="sum")],df)

LuxWidget(current_vis={'config': {'view': {'continuousWidth': 400, 'continuousHeight': 300}, 'axis': {'labelCo…

<Vis  (x: SUM(Horsepower), y: Origin) mark: bar, score: 0.0 >

You can also work with collections of Visualization via a `VisList` object.

In [25]:
from lux.vis.VisList import VisList

For example, we can create a set of visualizations of Weight with respect to all other attributes, using the wildcard “?” symbol.

In [26]:
VisList(["Horsepower","?"],df)

LuxWidget(recommendations=[{'action': 'Vis List', 'description': 'Shows a vis list defined by the intent', 'vs…

[<Vis  (x: Horsepower      , y: Displacement    ) mark: scatter, score: 0.00 >,
 <Vis  (x: Horsepower      , y: MilesPerGal     ) mark: scatter, score: 0.00 >,
 <Vis  (x: Year            , y: MEAN(Horsepower)) mark: line   , score: 0.00 >,
 <Vis  (x: MEAN(Horsepower), y: Name            ) mark: bar    , score: 0.00 >,
 <Vis  (x: MEAN(Horsepower), y: Origin          ) mark: bar    , score: 0.00 >,
 <Vis  (x: MEAN(Horsepower), y: Cylinders       ) mark: bar    , score: 0.00 >,
 <Vis  (x: Horsepower      , y: Weight          ) mark: scatter, score: 0.00 >,
 <Vis  (x: Horsepower      , y: Acceleration    ) mark: scatter, score: 0.00 >]

For more resources about how to use Lux:
- ReadTheDoc documentation: https://lux-api.readthedocs.io/en/latest/
- Notebook tutorial series: https://github.com/lux-org/lux/tree/master/examples/tutorial
